[Reference](https://medium.com/@Rohan_Dutt/10-forecasting-models-used-for-revenue-sales-and-demand-prediction-in-industry-162be7ef02ea)

# 1. Ensemble Forecasting

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

# Example: forecasts from three models
data = pd.read_csv("model_forecasts.csv")
# Columns: 'ARIMA', 'Prophet', 'XGBoost', 'Actual'

# Simple average ensemble
data['Ensemble_avg'] = data[['ARIMA','Prophet','XGBoost']].mean(axis=1)

# Stacked regression ensemble
X = data[['ARIMA','Prophet','XGBoost']]
y = data['Actual']
stack_model = LinearRegression()
stack_model.fit(X, y)
data['Ensemble_stacked'] = stack_model.predict(X)

print(data[['Actual','Ensemble_avg','Ensemble_stacked']].head())

# 2. Causal Impact (Bayesian Structural Time-Series)

In [2]:
import pandas as pd
from causalimpact import CausalImpact

# Example: daily sales before & after campaign
data = pd.read_csv("sales_campaign.csv", parse_dates=["Date"], index_col="Date")
pre_period = [0, 29]  # first 30 days
post_period = [30, 59]  # next 30 days after campaign

ci = CausalImpact(data, pre_period, post_period)
print(ci.summary())
ci.plot()

# 3. Dynamic Regression

In [3]:
import pandas as pd
import statsmodels.api as sm

# Example: Daily sales and temperature data
data = pd.read_csv("sales_weather.csv", parse_dates=["Date"], index_col="Date")
y = data["Sales"]
X = data[["Temperature", "Promotion"]]

# Add intercept
X = sm.add_constant(X)

# Fit dynamic regression (ARIMAX)
model = sm.tsa.SARIMAX(y, exog=X, order=(1,0,0))
fit = model.fit()
print(fit.summary())

# Forecast next 7 days with future predictors
future_X = pd.DataFrame({
    "const": 1,
    "Temperature": [22, 23, 21, 20, 22, 24, 23],
    "Promotion": [0,1,0,0,1,0,1]
})
forecast = fit.get_forecast(steps=7, exog=future_X)
print(forecast.predicted_mean)

# 4. Bass Diffusion Model

In [4]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

# Bass model function
def bass_model(t, p, q, m):
    return m * ((1 - np.exp(-(p+q)*t)) / (1 + (q/p)*np.exp(-(p+q)*t)))

# Example: hypothetical adoption data
t = np.arange(1, 21)  # 20 weeks
adopters = np.array([2, 5, 12, 20, 35, 50, 70, 95, 120, 150, 180, 210, 240, 270, 295, 320, 340, 355, 370, 380])

# Fit Bass model
params, _ = curve_fit(bass_model, t, adopters, bounds=(0, [1,1,1000]))
p, q, m = params
print(f"p={p:.3f}, q={q:.3f}, total market={m:.0f}")

# Forecast next 10 weeks
t_future = np.arange(21, 31)
forecast = bass_model(t_future, p, q, m)

plt.plot(t, adopters, 'o', label='Observed')
plt.plot(np.concatenate([t, t_future]), np.concatenate([adopters, forecast]), '-', label='Bass Forecast')
plt.xlabel("Weeks")
plt.ylabel("Cumulative Adopters")
plt.title("Bass Diffusion Forecast")
plt.legend()
plt.show()

# 5. Market Mix Modeling (MMM)

In [5]:
import pandas as pd
import statsmodels.api as sm

# Example: Monthly spend & sales
data = pd.read_csv("marketing_sales.csv")
X = data[["TV", "Google_Ads", "Promo"]]
y = data["Sales"]

# Add intercept
X = sm.add_constant(X)

# Fit linear regression
model = sm.OLS(y, X).fit()
print(model.summary())

# Bayesian update (simplified)
from pydataset import data as pydataset_data
import pymc as pm

with pm.Model() as bayes_model:
    alpha = pm.Normal("alpha", mu=0, sigma=10)
    betas = pm.Normal("betas", mu=0, sigma=5, shape=X.shape[1]-1)
    sigma = pm.HalfNormal("sigma", sigma=10)
    mu = alpha + pm.math.dot(X.iloc[:,1:], betas)
    y_obs = pm.Normal("y_obs", mu=mu, sigma=sigma, observed=y)
    trace = pm.sample(1000, tune=1000, cores=1, progressbar=False)

pm.summary(trace, var_names=["alpha","betas","sigma"])

# 6. Monte Carlo Simulation

In [6]:
import numpy as np
import matplotlib.pyplot as plt

# Example: simulate 12 months of revenue forecasts
np.random.seed(42)
mean_revenue = 50000      # base monthly revenue
std_dev = 8000            # volatility
n_simulations = 10000

# Run simulations
simulated_totals = np.random.normal(mean_revenue, std_dev, n_simulations)

# Probability of exceeding target
target = 55000
prob = np.mean(simulated_totals > target)
print(f"Probability revenue exceeds ${target:,}: {prob:.2%}")

# Visualize distribution
plt.hist(simulated_totals, bins=50, alpha=0.7)
plt.axvline(target, color='red', linestyle='--')
plt.title("Monte Carlo Revenue Distribution")
plt.xlabel("Simulated Monthly Revenue")
plt.ylabel("Frequency")
plt.show()

# 7. Prophet by Facebook
Prophet simplifies time-series forecasting with automatic seasonality detection and intuitive parameters.

# 8. XGBoost + LSTMs (Machine Learning Technique)

In [7]:
import pandas as pd
import xgboost as xgb
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.model_selection import train_test_split

# Example: price, weather, and past sales data
data = pd.read_csv("sales_with_weather.csv")
X = data[["Price", "Temperature", "Promotion"]]
y = data["Sales"]

# Step 1: XGBoost for external features
xgb_model = xgb.XGBRegressor(n_estimators=200, learning_rate=0.05)
xgb_model.fit(X, y)

# Step 2: LSTM for time-based patterns
seq = y.values[-100:].reshape(1, 100, 1)  # last 100 days
model = Sequential([
    LSTM(64, activation='relu', input_shape=(100, 1)),
    Dense(1)
])
model.compile(optimizer='adam', loss='mse')
model.fit(seq, y.values[-100:].reshape(1, 1), epochs=20, verbose=0)

# Forecast next step
future = model.predict(seq)
print(f"Next predicted sales: {future.flatten()[0]:.2f}")

# 9. Exponential Smoothing (ETS)

In [8]:
import pandas as pd
from statsmodels.tsa.holtwinters import ExponentialSmoothing

# Example: Daily sales data
data = pd.read_csv("daily_sales.csv", parse_dates=["Date"], index_col="Date")

# Fit ETS model with trend and seasonality
model = ExponentialSmoothing(
    data["Sales"],
    trend="add",
    seasonal="add",
    seasonal_periods=7  # weekly seasonality
)
fit = model.fit()

# Forecast next 14 days
forecast = fit.forecast(steps=14)
print(forecast)

# 10. Time Series Analysis (ARIMA)

In [9]:
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA
from pmdarima import auto_arima

# Example: Monthly retail sales data
data = pd.read_csv("retail_sales.csv", parse_dates=["Month"], index_col="Month")

# Auto-select best (p,d,q) parameters
auto_model = auto_arima(data["Sales"], seasonal=False, trace=True)
print(auto_model.summary())

# Fit final ARIMA model
model = ARIMA(data["Sales"], order=auto_model.order)
fit = model.fit()

# Forecast next 6 months
forecast = fit.forecast(steps=6)
print(forecast)